# Recommendation Systems

#### Two types of Recommendation Systems:
1. Collabative systems: Predict what we like based on other similars users has liked in the past
2. Content based systems: Predict what we like based on what we liked in the past.

Some system use both: like netflix


# Lets build our own Recommendation System

### Lets Install our dependency

In [42]:
import numpy as np
import scipy as sy
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

### Fetch Data and format it

In [43]:
data=fetch_movielens(min_rating=4.0)
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 49906 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 5469 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}

### Print Training and testing Data

In [44]:
print(repr(data["train"]))
print(repr(data["test"]))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


### Create model


Warp: weighted approximate rank pairwise . Warp helps to create recomm for  each user by looking at the existing user raiting pairs and predciting ranking for each.Uses Gradient decient to find the weight that improve our prediction over time.This takes both the user past raiting history content based and similar user ratings collaborative.Its a hybrid systems.

In [45]:
model = LightFM(loss='warp') 
#Tain our model
model.fit(data["train"],epochs=30,num_threads=2) #epochs--run and# num_threads --parallel computation 

# Generate recommendation for our model

In [46]:
def sample_recommendation(model, data, user_ids):

    #number of users and movies in training data
    n_users, n_items = data['train'].shape

    #generate recommendations for each user we input
    for user_id in user_ids:

        #movies they already like
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        #movies our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))
        #rank them in order of most liked to least
        top_items = data['item_labels'][np.argsort(-scores)]

        #print out the results
        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)
            
sample_recommendation(model, data, [3, 25, 450])


User 3
     Known positives:
        Seven (Se7en) (1995)
        Contact (1997)
        Starship Troopers (1997)
     Recommended:
        Scream (1996)
        Contact (1997)
        Starship Troopers (1997)
User 25
     Known positives:
        Dead Man Walking (1995)
        Star Wars (1977)
        Fargo (1996)
     Recommended:
        L.A. Confidential (1997)
        Fargo (1996)
        English Patient, The (1996)
User 450
     Known positives:
        Contact (1997)
        George of the Jungle (1997)
        Event Horizon (1997)
     Recommended:
        Devil's Advocate, The (1997)
        Saint, The (1997)
        Contact (1997)
